In [14]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import sparse
from skimage import transform
import ipywidgets as widgets

In [15]:
rsp = np.load('../../Data/simple_pCT/Phantoms/Head/RSP.npy')

scale = 1
phantom = np.pad(rsp[:,:,rsp.shape[2]//2], ((30*scale,30*scale), (100*scale, 100*scale)))
RSP_shape = phantom.shape[:2]

In [16]:
num_angle = 180
num_offset = 1
num_spotx = 130
chord_length = True
filter_name='ramp'
chords = 'exact' if chord_length else 'map'

MLP_angles_spotx = sparse.load_npz('../../Data/simple_pCT/MLP/MLP_angles{:d}_offset{:d}_spotx{:d}_exact_{:d}_{:d}.npz'.format(num_angle, num_offset, num_spotx, RSP_shape[0], RSP_shape[1]))

In [17]:
theta = np.linspace(0., 180., num_angle, endpoint=False)

reconstructed = np.empty((num_spotx,num_spotx,rsp.shape[2]))
for z in range(rsp.shape[2]):
    scale = 1
    phantom = np.pad(rsp[:,:,z], ((30*scale,30*scale), (100*scale, 100*scale)))

    x = phantom.flatten()

    b_w = MLP_angles_spotx @ x
    wepl = np.reshape(b_w, (num_spotx, num_angle))

    reconstructed[:,:,z] = transform.iradon(wepl, theta=theta, filter_name=filter_name, circle=True)


In [18]:
@widgets.interact(z=(0,rsp.shape[2]-1,1))
def show_reconstruction(z=0):
    plt.figure()
    plt.imshow(reconstructed[:,:,z], aspect='auto')
    plt.colorbar()
    plt.show()

interactive(children=(IntSlider(value=0, description='z', max=1025), Output()), _dom_classes=('widget-interact…

In [20]:
np.save('../../Data/simple_pCT/Reconstruction/Head/{:d}_{:d}/3D/RSP_angles{:d}_offset{:d}_spotx{:d}_{:s}_{:s}.npy'.format(RSP_shape[0], RSP_shape[1], num_angle, num_offset, num_spotx, chords, filter_name), reconstructed)